### Imports

In [8]:
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary

import pyLDAvis.gensim_models

import numpy as np
import logging
import warnings
warnings.filterwarnings('ignore')

from numpy import array

from nlp import *

### Setting up Logger

In [9]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


### Set up corpus

In [10]:
texts = [['human', 'interface', 'computer'],
         ['survey', 'user', 'computer', 'system', 'response', 'time'],
         ['eps', 'user', 'interface', 'system'],
         ['system', 'human', 'system', 'eps'],
         ['user', 'response', 'time'],
         ['trees'],
         ['graph', 'trees'],
         ['graph', 'minors', 'trees'],
         ['graph', 'minors', 'survey']]

vocab = Dictionary(texts)
bow_corpus = [vocab.doc2bow(text) for text in texts]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>
INFO:gensim.corpora.dictionary:built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
DEBUG:gensim.utils:starting a new internal lifecycle event log for Dictionary
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2023-01-31T12:29:20.027856', 'gensim': '4.3.0', 'python': '3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:26:04) [GCC 10.4.0]', 'platform': 'Linux-5.15.0-58-generic-x86_64-with-glibc2.35', 'event': 'created'}


### train LDA models

In [11]:
goodLdaModel = LdaModel(corpus=bow_corpus, id2word=vocab, iterations=50, num_topics=2)
badLdaModel = LdaModel(corpus=bow_corpus, id2word=vocab, iterations=1, num_topics=2)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.5
INFO:gensim.models.ldamodel:using symmetric eta at 0.5
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 9 documents, updating model once every 9 documents, evaluating perplexity every 9 documents, iterating 50x with a convergence threshold of 0.001000
DEBUG:gensim.models.ldamodel:bound: at document #0
INFO:gensim.models.ldamodel:-3.305 per-word bound, 9.9 perplexity estimate based on a held-out corpus of 9 documents with 29 words
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #9/9
DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:2/9 documents converged within 50 iterations
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #0 (0.500): 0.149*"graph" + 0.116*"minors" + 0.091*"human" + 0.090*"trees" + 0.082

### Get Coherence Model - `u_mass`

In [12]:
good_cm = CoherenceModel(model=goodLdaModel, corpus=bow_corpus, dictionary=vocab, coherence='u_mass')
bad_cm = CoherenceModel(model=badLdaModel, corpus=bow_corpus, dictionary=vocab, coherence='u_mass')

DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel<num_terms=12, num_topics=2, decay=0.5, chunksize=2000>
DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel<num_terms=12, num_topics=2, decay=0.5, chunksize=2000>


### Get Coherence Model - `c_v`

In [13]:
good_cm = CoherenceModel(model=goodLdaModel, texts=texts, dictionary=vocab, coherence='c_v')
bad_cm = CoherenceModel(model=badLdaModel, texts=texts, dictionary=vocab, coherence='c_v')

DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel<num_terms=12, num_topics=2, decay=0.5, chunksize=2000>
DEBUG:gensim.models.coherencemodel:Setting topics to those of the model: LdaModel<num_terms=12, num_topics=2, decay=0.5, chunksize=2000>


### Print Coherence Values

In [ ]:
print(good_cm.get_coherence())
print(bad_cm.get_coherence())

### Visualise Model

In [15]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(goodLdaModel, bow_corpus, vocab)

DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:9/9 documents converged within 50 iterations


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.017148  0.0       1        1  61.075982
0     -0.017148  0.0       2        1  38.924018, topic_info=         Term      Freq     Total Category  logprob  loglift
10      graph  2.000000  2.000000  Default  12.0000  12.0000
11     minors  2.000000  2.000000  Default  11.0000  11.0000
1       human  2.000000  2.000000  Default  10.0000  10.0000
2   interface  2.000000  2.000000  Default   9.0000   9.0000
0    computer  2.000000  2.000000  Default   8.0000   8.0000
4      survey  2.000000  2.000000  Default   7.0000   7.0000
9       trees  2.000000  2.000000  Default   6.0000   6.0000
8         eps  2.000000  2.000000  Default   5.0000   5.0000
5      system  3.000000  3.000000  Default   4.0000   4.0000
7        user  2.000000  2.000000  Default   3.0000   3.0000
6        time  2.000000  2.000000  Default   2.0000   2.0000
3    response  2.000000  2.000000  Default   1.0000   1.0000
7        user  2.143206  2.837269   Topic1  -2.1119   0.2125
5      system  2.663824  3.546294   Topic1  -1.8945   0.2069
6        time  1.579437  2.127414   Topic1  -2.4172   0.1952
3    response  1.470960  2.125323   Topic1  -2.4883   0.1250
8         eps  1.380305  2.123576   Topic1  -2.5519   0.0623
9       trees  1.812078  2.830886   Topic1  -2.2798   0.0469
4      survey  1.224718  2.120578   Topic1  -2.6715  -0.0559
0    computer  1.216276  2.120415   Topic1  -2.6785  -0.0628
2   interface  1.197058  2.120044   Topic1  -2.6944  -0.0785
1       human  1.085927  2.117904   Topic1  -2.7918  -0.1749
10      graph  1.136626  2.817873   Topic1  -2.7462  -0.4149
11     minors  0.801620  2.112424   Topic1  -3.0954  -0.4759
11     minors  1.310805  2.112424   Topic2  -2.1531   0.4664
10      graph  1.681247  2.817873   Topic2  -1.9042   0.4271
1       human  1.031977  2.117904   Topic2  -2.3923   0.2246
2   interface  0.922986  2.120044   Topic2  -2.5039   0.1120
0    computer  0.904139  2.120415   Topic2  -2.5245   0.0912
4      survey  0.895860  2.120578   Topic2  -2.5337   0.0819
9       trees  1.018809  2.830886   Topic2  -2.4051  -0.0784
8         eps  0.743271  2.123576   Topic2  -2.7204  -0.1062
3    response  0.654362  2.125323   Topic2  -2.8478  -0.2345
6        time  0.547977  2.127414   Topic2  -3.0253  -0.4129
5      system  0.882470  3.546294   Topic2  -2.5488  -0.4474
7        user  0.694064  2.837269   Topic2  -2.7889  -0.4645, token_table=      Topic      Freq       Term
term                            
0         1  0.471606   computer
0         2  0.471606   computer
8         1  0.470904        eps
8         2  0.470904        eps
10        1  0.354878      graph
10        2  0.709755      graph
1         1  0.472165      human
1         2  0.472165      human
2         1  0.471688  interface
2         2  0.471688  interface
11        1  0.473390     minors
11        2  0.473390     minors
3         1  0.470517   response
3         2  0.470517   response
4         1  0.471570     survey
4         2  0.471570     survey
5         1  0.845954     system
5         2  0.281985     system
6         1  0.940109       time
6         2  0.470054       time
9         1  0.706492      trees
9         2  0.353246      trees
7         1  0.704903       user
7         2  0.352452       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [16]:
pyLDAvis.gensim_models.prepare(badLdaModel, bow_corpus, vocab)

DEBUG:gensim.models.ldamodel:performing inference on a chunk of 9 documents
DEBUG:gensim.models.ldamodel:0/9 documents converged within 1 iterations


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
0      0.001247  0.0       1        1  52.270705
1     -0.001247  0.0       2        1  47.729295, topic_info=         Term      Freq     Total Category  logprob  loglift
2   interface  2.000000  2.000000  Default  12.0000  12.0000
8         eps  2.000000  2.000000  Default  11.0000  11.0000
10      graph  2.000000  2.000000  Default  10.0000  10.0000
5      system  3.000000  3.000000  Default   9.0000   9.0000
1       human  2.000000  2.000000  Default   8.0000   8.0000
9       trees  2.000000  2.000000  Default   7.0000   7.0000
7        user  2.000000  2.000000  Default   6.0000   6.0000
4      survey  2.000000  2.000000  Default   5.0000   5.0000
3    response  2.000000  2.000000  Default   4.0000   4.0000
0    computer  2.000000  2.000000  Default   3.0000   3.0000
6        time  2.000000  2.000000  Default   2.0000   2.0000
11     minors  2.000000  2.000000  Default   1.0000   1.0000
9       trees  1.641208  2.832237   Topic1  -2.2231   0.1031
3    response  1.207525  2.123751   Topic1  -2.5300   0.0841
0    computer  1.197906  2.123575   Topic1  -2.5380   0.0762
6        time  1.183030  2.123302   Topic1  -2.5505   0.0638
11     minors  1.141112  2.122535   Topic1  -2.5866   0.0281
4      survey  1.096297  2.121716   Topic1  -2.6266  -0.0116
7        user  1.461606  2.828953   Topic1  -2.3390  -0.0116
1       human  1.067280  2.121186   Topic1  -2.6534  -0.0381
5      system  1.759363  3.534953   Topic1  -2.1536  -0.0490
10      graph  1.390678  2.827653   Topic1  -2.3888  -0.0609
8         eps  1.007781  2.120097   Topic1  -2.7108  -0.0950
2   interface  1.004719  2.120041   Topic1  -2.7139  -0.0980
2   interface  1.115321  2.120041   Topic2  -2.5185   0.0973
8         eps  1.112316  2.120097   Topic2  -2.5212   0.0946
10      graph  1.436975  2.827653   Topic2  -2.2651   0.0627
5      system  1.775590  3.534953   Topic2  -2.0535   0.0511
1       human  1.053907  2.121186   Topic2  -2.5752   0.0402
7        user  1.367346  2.828953   Topic2  -2.3148   0.0126
4      survey  1.025419  2.121716   Topic2  -2.6026   0.0125
11     minors  0.981423  2.122535   Topic2  -2.6464  -0.0317
6        time  0.940272  2.123302   Topic2  -2.6893  -0.0749
0    computer  0.925669  2.123575   Topic2  -2.7049  -0.0907
3    response  0.916226  2.123751   Topic2  -2.7152  -0.1011
9       trees  1.191029  2.832237   Topic2  -2.4529  -0.1266, token_table=      Topic      Freq       Term
term                            
0         1  0.470904   computer
0         2  0.470904   computer
8         1  0.471677        eps
8         2  0.471677        eps
10        1  0.353650      graph
10        2  0.353650      graph
1         1  0.471434      human
1         2  0.471434      human
2         1  0.471689  interface
2         2  0.471689  interface
11        1  0.471135     minors
11        2  0.471135     minors
3         1  0.470865   response
3         2  0.470865   response
4         1  0.471317     survey
4         2  0.471317     survey
5         1  0.565778     system
5         2  0.565778     system
6         1  0.470965       time
6         2  0.470965       time
9         1  0.706156      trees
9         2  0.353078      trees
7         1  0.353488       user
7         2  0.353488       user, R=12, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])